In [45]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import csv
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time

In [46]:
test = []
names = []
flipPrices = []
prodNames = []
info,price = [],[]
url = "https://flipkart.com/search?q="
# query to search for.
q = input("Enter product name (Searching on flipkart)")
q = q.replace(" ","+")
#     q = input("Enter a query: ")
file_name = q.replace(" ", "_")
# response recieved in bytes
resp = requests.get(url+q)
# parsing response content using BeautifulSoup class, so that we can perform operations on it.
parsed_html = bs(resp.content, 'html.parser')
# data cleaning
raw_data = parsed_html.find("script", attrs={"id":"is_script"})
data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
json_data = json.loads(data)
req_data = json_data["pageDataV4"]["page"]["data"]["10003"]   #[10]["widget"]["data"]["products"][3]["productInfo"]
#req_json_data = json_data["seoMeta"]["answerBox"]["data"]["renderableComponents"][0]["value"]["data"]

In [47]:
data_list = []
# print(len(req_data))
try:
    for i in range(1, len(req_data)):
        d = {}
        jd = req_data[i]["widget"]["data"]["products"]
        # print(len(jd))
        # print("i: ", i, end="\n")
        for j in range(len(jd)):
            jd2 = jd[j]["productInfo"]["value"]

            d["title"] = jd2["titles"]["title"]
            d["keySpecs"] = jd2["keySpecs"]
            d["rating"] = jd2["rating"]["average"]
            d["ratingCount"] = jd2["rating"]["count"]
            d["price"] = jd2["pricing"]["finalPrice"]["value"]
            #d["warranty"] = jd2["warrantySummary"]
            d["url"] = jd2["smartUrl"]

            # You can uncomment below lines if you want to print json output on terminal

            # print("Title: ",jd2["titles"]["title"],end="\n")
            # print("key specs: ", jd2["keySpecs"], end="\n")
            # print("Rating: ", jd2["rating"]["average"], end="\n")
            # print("Total ratings: ", jd2["rating"]["count"], end="\n")
            # print("Price: ", jd2["pricing"]["finalPrice"]["value"],end="\n")
            # print("warranty: ", jd2["warrantySummary"], end="\n")
            # print("Smart url: ", jd2["smartUrl"], end="\n")
        data_list.append(d)
        
except:
    pass 
# dumping data to result.json file
    # print(list(data_list))
with open("flipkart"+'.json', 'w') as fp:
    json.dump(data_list, fp)

# Now let us write our data to csv file
data_file = open("flipkart"+'.csv', 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file) 

# Counter variable used for writing  
# headers to the CSV file 
count = 0




In [48]:
for data in data_list:
    if count == 0: 

        # Writing headers of CSV file 
        header = data.keys() 
        csv_writer.writerow(header) 
        count += 1
        
    # Writing data of CSV file 

    csv_writer.writerow(data.values()) 


with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        #name = " ".join(name.split(' ')[0:2])
        #print(name)
        #print("name = ",name)
        names.append(name)
        i += 1
        j += 2
    except:
            break


In [49]:
print("Best results",len(names))
#print(names,len(names))
if len(names) <= 10:
    flipkart_url = "https://www.flipkart.com/search?q=" + q
    print(flipkart_url)
    uClient = uReq(flipkart_url)
    flipkartPage = uClient.read()
    uClient.close()
    flipkart_html = bs(flipkartPage, "html.parser")
    bigboxes = flipkart_html.find_all("a", {"class": "s1Q9rs"})
    soup = BeautifulSoup(flipkartPage, 'html.parser')
    info = soup.select("[class~=s1Q9rs]")
    flipPrices = soup.select("[class =_30jeq3]")
    prodNames = [i.get('title') for i in info]
    names = prodNames
    df = pd.DataFrame(list(zip(prodNames, flipPrices)), 
                columns =['product_name', 'Flipkart_price']) 
    df.to_csv('test.csv')
    print(df)

else:
    with open('flipkart.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        rows = list(reader)

Best results 0
https://www.flipkart.com/search?q=4+GB+RAM+Laptop
                                         product_name Flipkart_price
0   SAMSUNG Galaxy Book Go Snapdragon 7c Gen 2 - (...      [₹32,990]
1   HP Athlon Dual Core 3050U - (4 GB/256 GB SSD/D...      [₹22,490]
2   DELL Vostro Core i3 11th Gen - (4 GB/1 TB HDD/...      [₹34,990]
3   Lenovo IdeaPad Core i3 10th Gen - (4 GB/256 GB...      [₹31,490]
4   ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...      [₹24,990]
5   ASUS VivoBook 15 (2022) Core i3 11th Gen - (4 ...      [₹34,990]
6   ASUS VivoBook 14 (2021) Celeron Dual Core - (4...      [₹26,990]
7   Infinix INBook X1 Neo Series Celeron Quad Core...      [₹22,990]
8   Lenovo ThinkPad E15 Core i3 11th Gen - (4 GB/2...      [₹44,700]
9   Lenovo Core i3 11th Gen - (4 GB/256 GB SSD/Win...      [₹38,990]
10  acer Extensa 15 Core i3 11th Gen - (4 GB/256 G...      [₹30,990]
11  acer Aspire 3 Dual Core 3020e - (4 GB/256 GB S...      [₹24,990]
12  acer Aspire 3 Ryzen 3 Dual Core R3

In [50]:
    i,j = 0,2
    while i in range(len(rows)):
        try:
            price = rows[j][4]
            #price = price[i].text
            #print("price = ",price)
            flipPrices.append(price)
            i += 1
            j += 2
        except:
            break
    df = pd.DataFrame(list(zip(names, flipPrices)), 
            columns =['Product_name', 'Flipkart_price'])

    print(df)
    df.to_csv('test.csv')

data_file.close()

                                         Product_name Flipkart_price
0   SAMSUNG Galaxy Book Go Snapdragon 7c Gen 2 - (...      [₹32,990]
1   HP Athlon Dual Core 3050U - (4 GB/256 GB SSD/D...      [₹22,490]
2   DELL Vostro Core i3 11th Gen - (4 GB/1 TB HDD/...      [₹34,990]
3   Lenovo IdeaPad Core i3 10th Gen - (4 GB/256 GB...      [₹31,490]
4   ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...      [₹24,990]
5   ASUS VivoBook 15 (2022) Core i3 11th Gen - (4 ...      [₹34,990]
6   ASUS VivoBook 14 (2021) Celeron Dual Core - (4...      [₹26,990]
7   Infinix INBook X1 Neo Series Celeron Quad Core...      [₹22,990]
8   Lenovo ThinkPad E15 Core i3 11th Gen - (4 GB/2...      [₹44,700]
9   Lenovo Core i3 11th Gen - (4 GB/256 GB SSD/Win...      [₹38,990]
10  acer Extensa 15 Core i3 11th Gen - (4 GB/256 G...      [₹30,990]
11  acer Aspire 3 Dual Core 3020e - (4 GB/256 GB S...      [₹24,990]
12  acer Aspire 3 Ryzen 3 Dual Core R33250U - (4 G...      [₹29,990]
13  ASUS Vivobook 14 Core i3 10th 

In [51]:
data = pd.read_csv("flipkart.csv", usecols = ['title','keySpecs','rating','ratingCount','price'])
data

,title,keySpecs,rating,ratingCount,price
0,ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...,"['Intel Celeron Dual Core Processor', '4 GB DD...",4.0,133,24990
1,Lenovo ThinkPad E15 Core i3 11th Gen - (4 GB/2...,"['Intel Core i3 Processor (11th Gen)', '4 GB D...",4.2,60,44700
2,acer Aspire 3 Ryzen 3 Dual Core R33250U - (4 G...,"['AMD Ryzen 3 Dual Core Processor', '4 GB DDR4...",4.1,41,29990
3,ASUS Chromebook Flip Touch Celeron Dual Core -...,"['Intel Celeron Dual Core Processor', '4 GB LP...",4.0,846,25990
4,Lenovo IdeaPad 3 Celeron Dual Core 4th Gen - (...,['Intel Celeron Dual Core Processor (4th Gen)'...,4.1,842,27990
5,ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...,"['Intel Celeron Dual Core Processor', '4 GB DD...",4.2,49,30490
6,ASUS EeeBook 12 Celeron Dual Core - (4 GB/64 G...,"['Intel Celeron Dual Core Processor', '4 GB DD...",4.5,11,21990
7,Lenovo IdeaPad 3 Celeron Dual Core - (4 GB/256...,"['Intel Celeron Dual Core Processor', '4 GB DD...",0.0,0,31000
8,ASUS VivoBook Ultra 14 Core i3 10th Gen - (4 G...,"['Intel Core i3 Processor (10th Gen)', '4 GB D...",4.4,4118,38990
9,DELL Inspiron Core i3 10th Gen - (4 GB/256 GB ...,"['Intel Core i3 Processor (10th Gen)', '4 GB D...",4.4,678,39852


In [52]:
import requests
import time

In [53]:
flipkart=''
amazon=''
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

amazonlist = []
amazonName = []
i = 0
while i in range(len(names)):
    print(names[i])
    def amazon(name):
            try:
                global amazon
                name = " ".join(name.split(' ')[0:2])
                name1 = name.replace(" ","-")
                name2 = name.replace(" ","+")
                amazon=f'https://www.amazon.in/{name1}/s?k={name2}'
                res = requests.get(f'https://www.amazon.in/{name1}/s?k={name2}',headers=headers)
                print("\nSearching in amazon:")
                soup = BeautifulSoup(res.text,'html.parser')
                amazon_page = soup.select('.a-color-base.a-text-normal')
                amazon_page_length = int(len(amazon_page))
                for i in range(0,amazon_page_length):
                    name = name.upper()
                    amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                    if name in amazon_name[0:20]:
                        amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                        amazon_price = soup.select('.a-price-whole')[i].getText().strip().upper()
                        amazonlist.append(amazon_price)
                        print("Amazon:")
                        print(amazon_name)
                        amazonName.append(amazon_name)
                        print("₹"+amazon_price)
                        print("-----------------------")
                        break
                    else:
                        i+=1
                        i=int(i)
                        if i==amazon_page_length:
                            print("amazon : No product found!")
                            print("-----------------------")
                            amazon_price = '0'
                            amazonlist.append(amazon_price)
                            amazonName.append("No similar product")
                            break
                
                return amazon_price
            except:
                print("amazon: No product found!")
                print("-----------------------")
                amazon_price = '0'
                amazonlist.append(amazon_price)
                amazonName.append("No similar product")
            return amazon_price
    amazon_price = amazon(names[i])
    flipkart=''
    ebay=''
    croma=''
    amazon=''
    olx=''
    i += 1
flip = flipPrices
idk = []
for i in range(len(flip)):
    try:
        x = flip[i].text.replace('₹','')
        print(x)
        idk.append(x)
    except:
        idk = test
df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)), 
               columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])
    
df.to_csv('flipkartandamazon.csv')
df

SAMSUNG Galaxy Book Go Snapdragon 7c Gen 2 - (4 GB/128 GB EMMC Storage/Windows 11 Home) NP340XLA-KA1IN Thin and Light Laptop

Searching in amazon:
Amazon:
SAMSUNG GALAXY M32 PRIME EDITION (LIGHT BLUE, 6GB RAM, 128GB)
₹34,990.
-----------------------
HP Athlon Dual Core 3050U - (4 GB/256 GB SSD/DOS) 255 G8 Notebook

Searching in amazon:
amazon : No product found!
-----------------------
DELL Vostro Core i3 11th Gen - (4 GB/1 TB HDD/256 GB SSD/Windows 11 Home) Vostro 3400 Thin and Light Laptop

Searching in amazon:
Amazon:
DELL VOSTRO 3400 LAPTOP, INTEL I5-1135G7, 8GB DDR4, 512GB SSD, NVIDIA MX330 2GB GDDR5, WIN 11 + OFFICE H&S 2021, 14.0" (35.56 CMS) FHD WVA AG, DUNE COLOR (D552304WIN9DE, 1.59KGS)
₹56,700
-----------------------
Lenovo IdeaPad Core i3 10th Gen - (4 GB/256 GB SSD/Windows 11 Home) 15IIL05 Thin and Light Laptop

Searching in amazon:
Amazon:
LENOVO IDEAPAD GAMING 3 AMD RYZEN 5 5600H 15.6" (39.62CM) FHD IPS GAMING LAPTOP (8GB/512GB SSD/4GB NVIDIA GTX 1650/120HZ/WIN 11/BACKLI

,Product_name_Flipkart,Flipkart_price,Product_name_Amazon,Amazon_price
0,SAMSUNG Galaxy Book Go Snapdragon 7c Gen 2 - (...,"32,990","SAMSUNG GALAXY M32 PRIME EDITION (LIGHT BLUE, ...","34,990."
1,HP Athlon Dual Core 3050U - (4 GB/256 GB SSD/D...,"22,490",No similar product,0
2,DELL Vostro Core i3 11th Gen - (4 GB/1 TB HDD/...,"34,990","DELL VOSTRO 3400 LAPTOP, INTEL I5-1135G7, 8GB ...","56,700"
3,Lenovo IdeaPad Core i3 10th Gen - (4 GB/256 GB...,"31,490",LENOVO IDEAPAD GAMING 3 AMD RYZEN 5 5600H 15.6...,"80,990"
4,ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...,"24,990",No similar product,0
5,ASUS VivoBook 15 (2022) Core i3 11th Gen - (4 ...,"34,990","ASUS VIVOBOOK 15, 15.6-INCH (39.62 CMS) FHD, A...","72,990"
6,ASUS VivoBook 14 (2021) Celeron Dual Core - (4...,"26,990","ASUS VIVOBOOK 15, 15.6-INCH (39.62 CMS) FHD, A...","40,528"
7,Infinix INBook X1 Neo Series Celeron Quad Core...,"22,990",INFINIX INBOOK INTEL X1 CORE I5 10TH GEN - (8 ...,"27,900"
8,Lenovo ThinkPad E15 Core i3 11th Gen - (4 GB/2...,"44,700","LENOVO THINKPAD E14 AMD RYZEN 3 5300U 14"" (35....","44,715"
9,Lenovo Core i3 11th Gen - (4 GB/256 GB SSD/Win...,"38,990",No similar product,0
